In [4]:
import os 
from langdetect import detect
import edge_tts
from pydantic import BaseModel
import uuid
import pathlib
from google.cloud import storage
from urllib.parse import urlparse
from google.cloud import exceptions

In [5]:
class TextRequest(BaseModel):
    text: str

In [ ]:
class GCStorage:
    def __init__(self, credentials_path: str | None = None):
        if credentials_path:
            os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

        self.client = storage.Client()

    def upload_blob(self, bucket_name, src_file_name):
        bucket = self.client.bucket(bucket_name)
        blob_name = os.path.basename(src_file_name)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(src_file_name)
        blob.make_public()
        return blob.public_url
    
    def delete_blob(self, url: str):
        """
        Delete a blob from its public URL or gs:// path.
        Args:
            url: The public URL (https://storage.googleapis.com/...) or gs:// path.
        """
        try:
            # Parse bucket name and blob name from URL
            if url.startswith("gs://"):
                # Example: gs://my-bucket/tts/audio.mp3
                parts = url.replace("gs://", "").split("/", 1)
                bucket_name, blob_name = parts[0], parts[1]
            else:
                # Example: https://storage.googleapis.com/my-bucket/tts/audio.mp3
                parsed = urlparse(url)
                parts = parsed.path.lstrip("/").split("/", 1)
                bucket_name, blob_name = parts[0], parts[1]

            bucket = self.client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            blob.delete()

        except exceptions.NotFound:
            print(f"File not found: gs://{bucket_name}/{blob_name}")

        except Exception as e:
            print(f"Delete error: {e}")


In [18]:
a = GCStorage(credentials_path="./GCP/rare-karma-468001-k3-a7efcf8a7c80.json")
a.upload_blob("tts-script" ,"./GCP/1.jpg")

'https://storage.googleapis.com/tts-script/1.jpg'

In [20]:
a.delete_blob('https://storage.googleapis.com/tts-script/1.jpg')

NotFound: 404 DELETE https://storage.googleapis.com/storage/v1/b/tts-script/o/1.jpg?prettyPrint=false: No such object: tts-script/1.jpg

In [ ]:
import asyncio
import ssl
import edge_tts

# WARNING: INSECURE - Disables all SSL certificate verification
# This should only be used for temporary debugging.
ssl._create_default_https_context = ssl._create_unverified_context

async def test():
    print("Attempting to connect to Edge TTS service...")
    try:
        tts = edge_tts.Communicate("Hello world", "en-US-JennyNeural")
        await tts.save("test.mp3")
        print("Successfully created test.mp3!")
    except Exception as e:
        print(f"An error occurred: {e}")

# This part remains the same
try:
    loop = asyncio.get_running_loop()
    if loop.is_running():
        # If already running (like in Jupyter/IPython)
        task = loop.create_task(test())
    else:
        asyncio.run(test())
except RuntimeError:
    # For standard Python scripts
    asyncio.run(test())

Attempting to connect to Edge TTS service...
An error occurred: Cannot connect to host api.msedgeservices.com:443 ssl:True [SSLCertVerificationError: ('A required certificate is not within its validity period when verifying against the current system clock or the timestamp in the signed file.',)]


In [1]:
import ssl, certifi
print("Using:", ssl.get_default_verify_paths().cafile)
print("Certifi:", certifi.where())


Using: D:\App\Miniconda\envs\chatbot\Lib\site-packages\certifi\cacert.pem
Certifi: d:\App\Miniconda\envs\chatbot\Lib\site-packages\certifi\cacert.pem


In [3]:
async def text_to_speech(req: TextRequest):
    text = req.text.strip()
    if not text:
        return {"error": "Text cannot be empty"}
    if len(text) > 1000:
        return {"error": "Text too long (max 1000 chars)"}

    # Detect language and choose voice
    lang_code = detect(text)
    voice = f"en-US-JennyNeural"  # fallback default
    try:
        voices = await edge_tts.list_voices()
        candidates = [v for v in voices if v["Locale"].startswith(lang_code)]
        if candidates:
            voice = candidates[0]["Name"]
    except Exception:
        pass

    # Prepare file path
    os.makedirs("outputs", exist_ok=True)
    filename = f"tts_{uuid.uuid4().hex[:8]}.mp3"
    filepath = os.path.join("outputs", filename)

    # Generate TTS to file
    communicate = edge_tts.Communicate(
        text=text,
        voice=voice,
        rate="-5%",
        volume="+5%",
    )
    await communicate.save(filepath)

    return {
        "status": "success",
        "language_detected": lang_code,
        "voice_used": voice,
        "file_path": filepath,
    }

NameError: name 'TextRequest' is not defined

In [ ]:
text_to_speech("")